### 自定义Model

通过继承tf.keras.Model自定义模型，定义自己的前向传播来构建完全可自定义的模型。  
在 \_\_init\_\_ 方法中创建层并将它们设置为类实例的属性。  
在 call 方法中定义前向传播。  

在启用 Eager Execution 时，模型子类化特别有用，因为可以命令式地编写前向传播。

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

import numpy as np

ImportError: numpy.core.multiarray failed to import

ImportError: numpy.core.multiarray failed to import

ImportError: numpy.core.umath failed to import

ImportError: numpy.core.umath failed to import

AttributeError: module 'numpy' has no attribute 'integer'

In [9]:
# 创建数据集
data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

In [5]:
class MyModel(tf.keras.Model):  # 继承tf.keras.Model父类

  def __init__(self, num_classes=10):
    super(MyModel, self).__init__(name='my_model')
    self.num_classes = num_classes
    # 定义layers.
    self.dense_1 = layers.Dense(32, activation='relu')
    self.dense_2 = layers.Dense(num_classes, activation='sigmoid')

  def call(self, inputs):
    # 定义前向传播，使用`__init__`中定义的layers
    x = self.dense_1(inputs)
    return self.dense_2(x)

In [6]:
# 实例化自定义模型
model = MyModel(num_classes=10)

In [7]:
# 通过调用 compile 方法配置该模型的学习流程
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
# 模型训练
model.fit(data, labels, batch_size=32, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 203us/sample - loss: 11.4819 - accuracy: 0.1040
Epoch 2/5
1000/1000 [==============================] - 0s 30us/sample - loss: 11.4588 - accuracy: 0.1050
Epoch 3/5
1000/1000 [==============================] - 0s 30us/sample - loss: 11.4544 - accuracy: 0.1030
Epoch 4/5
1000/1000 [==============================] - 0s 27us/sample - loss: 11.4513 - accuracy: 0.1130
Epoch 5/5
1000/1000 [==============================] - 0s 27us/sample - loss: 11.4486 - accuracy: 0.1110


### 自定义Layer

通过继承tf.keras.layers.Layer创建子类，并实现以下方法来创建自定义层：

\_\_init\_\_: (可选)定义sublayers，可用于下面定义layer。   

build：创建层的权重。使用 add_weight 方法添加权重。    
call：定义前向传播。  
compute_output_shape：指定在给定输入形状的情况下如何计算层的输出形状。  
或者，可以通过实现 get_config 方法和 from_config 类方法序列化层。  
下面是一个使用核矩阵实现输入 matmul 的自定义层示例：

In [12]:
class MyLayer(layers.Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel',
                                  shape=(input_shape[1], self.output_dim),
                                  initializer='uniform',
                                  trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)

    def get_config(self):
        base_config = super(MyLayer, self).get_config()
        base_config['output_dim'] = self.output_dim
        return base_config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

使用自定义层创建模型

In [13]:
model = tf.keras.Sequential([
    MyLayer(10),
    layers.Activation('softmax')])

# The compile step specifies the training configuration
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Trains for 5 epochs.
model.fit(data, labels, batch_size=32, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 71us/sample - loss: 11.4897 - accuracy: 0.0890
Epoch 2/5
1000/1000 [==============================] - 0s 24us/sample - loss: 11.4896 - accuracy: 0.0890
Epoch 3/5
1000/1000 [==============================] - 0s 24us/sample - loss: 11.4894 - accuracy: 0.0850
Epoch 4/5
1000/1000 [==============================] - 0s 19us/sample - loss: 11.4886 - accuracy: 0.0850
Epoch 5/5
1000/1000 [==============================] - 0s 21us/sample - loss: 11.4891 - accuracy: 0.0900


### 自定义callbacks